In [ ]:
#load libraries
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
file_path = '/content/drive/MyDrive/673Project/Datasets/Symptom2Disease.csv'
file_path_attack_dataset = '/content/drive/MyDrive/673Project/Datasets/IrrelevantSymptoms_100.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive


In [ ]:

#change the file path
df = pd.read_csv(file_path)
print(len(df))
#print(df.head())
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
#print(shuffled_df.head())

1200


In [ ]:
X = list(df['text'])
Y = df['label']
#Here X and Y is the clinical note and label respectively

In [ ]:

#create on ehot encoding
unique_labels = list(set(Y))

one_hot_encoded = np.zeros((len(Y), len(unique_labels)))
print(type(one_hot_encoded[0][0]))
for i, label in enumerate(Y):

    one_hot_encoded[i][unique_labels.index(label)] = 1


<class 'numpy.float64'>


In [ ]:
#create train and test set
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_encoded, test_size=0.2, random_state=42)


In [ ]:
# Load ClinicalBERT
model_name = "emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
#create Clinical Bert dataset class
class ClinicalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize text
        inputs = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.float),
            #"label_index":torch.tensor(label, dtype=torch.long),
        }



# Create dataset and DataLoader
dataset = ClinicalDataset(X_train, y_train, tokenizer)
#print(dataset[0])
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
#crete testing dataset of clinical bert
dataset_test = ClinicalDataset(X_test, y_test, tokenizer)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=True)

true_labels_test = np.argmax(y_test, axis=1)
print(true_labels_test)
actual_labels_text = [unique_labels[i] for i in true_labels_test]


[20 18  8 13  6  2 16 11 13 18 14  9  9 14  7  4  3 21 14 13 15  5  0 16
  8 10  0 23 15 23 15 18  0  1 15 23 13  6 16  0 19  8  2 15 11 16 15  9
  9  7  1  2 15  4  2 16 11 20 12 15 16  4  7  6  4 11  6 13  3 17 13  8
 20  6  6 13  4  6 22 18 20  7 14  8  3 12 19  4 12  5  3 17 10 15 12  0
  3 22 10  8 23  8  1  6 18  1  2 22 20 22 15 18  4 11  5 12  7  4  1  4
  3 22 12 19 13 20 16  8  0  5 10  2 16  7 12  5  5 17 17 22 19  5 15  5
 21  3  1  0 23 17  2 16  1  7 22 13 16 10  2 16 19 22 12 10 10 14  6 18
 14  9 21 11 16 10  4 23  4  2  6 21  1 14  7 12 10 11 17 14 13  3 14  0
  9 14 17  4  6 22 11 10 14 22  0 11 18  7 21  1  1  2 20 21 13 18 22 10
  5  7  0 23  2 11 12 11 10  3 21  6  7  5 16  8 12  5 11 15 18  3 20 18]


In [ ]:
print(dataloader_test)

In [ ]:
#create classifier
class ClinicalBERTClassifier(nn.Module):
    def __init__(self, clinical_bert, num_classes):
        super(ClinicalBERTClassifier, self).__init__()
        self.bert = clinical_bert
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        # Pass inputs through ClinicalBERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # Extract the [CLS] token representation
        cls_output = outputs.last_hidden_state[:, 0, :]
        # Apply dropout and classification layer
        logits = self.classifier(self.dropout(cls_output))
        logits = F.softmax(logits, dim=1)
        #return self.bert.config.hidden_size
        #logits = self.classifier(cls_output)
        return logits



In [ ]:
num_classes = len(unique_labels)
num_epochs = 13;
batch_size = 16
learning_rate = 2e-5
model_classifier = ClinicalBERTClassifier(model, num_classes)
optimizer = Adam(model_classifier.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
#train the model for certain epochs
for epoch in range(num_epochs):
    model_classifier.train()
    total_loss = 0
    for batch in dataloader:

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        # Forward pass
        optimizer.zero_grad()
        logits = model_classifier(input_ids, attention_mask)
        loss = criterion(logits, labels)
        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        dbg = 1
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")


KeyboardInterrupt: 

In [ ]:
#evaluation of the trained model
def evaluate_model(model_classifier, dataloader_test):
    model_classifier.eval()
    predictions, probabilities, true_labels = [], [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = batch["label"]

            logits = model_classifier(input_ids, attention_mask)
            probs = F.softmax(logits, dim=1)  # Apply softmax to logits
            preds = torch.argmax(probs, dim=1)

            probabilities.extend(probs.numpy())
            predictions.extend(preds.numpy())
            true_labels.extend(np.argmax(labels.numpy(),axis =1))
            #print(true_labels)
            #sys.exit()
            #true_labels_test = np.argmax(y_test, axis=1)


    print(len(predictions))
    print(len(true_labels))
    #accuracy = (predictions == true_labels).sum().item() / true_labels.size(0)
    accuracy = sum(1 for a, b in zip(predictions, true_labels) if a == b)
    accuracy = accuracy/len(true_labels)
    print(accuracy)
    return predictions, probabilities, true_labels

# Evaluate model
preds, probs, true_labels = evaluate_model(model_classifier, dataloader_test)

In [ ]:
import joblib

In [ ]:

joblib.dump(model_classifier, '/content/drive/MyDrive/673Project/ClinicalBert/model_file.pkl')

['/content/drive/MyDrive/673Project/ClinicalBert/model_file.pkl']

In [ ]:
#model_classifer = joblib.load('/content/drive/MyDrive/673Project/ClinicalBert/model_file.pkl')

AttributeError: Can't get attribute 'ClinicalBERTClassifier' on <module '__main__'>

In [ ]:

df = pd.read_csv(file_path_attack_dataset )
print(len(df))
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)


100


In [ ]:
original_text = df['Original_text']
attacks_text = df['Attack_text']
print(len(original_text))

100


In [ ]:
#attack dataset class
class AttackDataset(Dataset):
    def __init__(self, original_text, attack_text, tokenizer, max_length=128):
        self.original_texts = original_text
        self.attack_texts = attack_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.original_texts)

    def __getitem__(self, idx):
        text = self.original_texts[idx]
        attack = self.attack_texts[idx]
        # Tokenize text
        inputs_original = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        inputs_attack = self.tokenizer(
            attack,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": inputs_original["input_ids"].squeeze(0),
            "attention_mask": inputs_original["attention_mask"].squeeze(0),
            "input_ids_attack": inputs_attack["input_ids"].squeeze(0),
            "attention_mask_attack": inputs_attack["attention_mask"].squeeze(0),
            #"label_index":torch.tensor(label, dtype=torch.long),
        }


dataset_attack = AttackDataset(original_text, attacks_text, tokenizer)
print(len(dataset_attack))
dataloader_attack = DataLoader(dataset_attack, batch_size=32, shuffle=True)

100


In [ ]:
#evaluation attacks
def evaluate_model(model_classifier, dataloader_test):
    model_classifier.eval()
    predictions_without_attack, probabilities_without_attack = [], []
    predictions_with_attack, probabilities_with_attack = [], []
    with torch.no_grad():
        for batch in dataloader_attack:
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            input_ids_attack = batch["input_ids_attack"]
            attention_mask_attack = batch["attention_mask_attack"]
            #labels = batch["label"]

            logits_without_attack = model_classifier(input_ids, attention_mask)
            logits_with_attack = model_classifier(input_ids_attack, attention_mask_attack)
            probs_without_attack = F.softmax(logits_without_attack, dim=1)  # Apply softmax to logits
            probs_with_attack = F.softmax(logits_with_attack, dim=1)
            #preds_without = torch.argmax(probs, dim=1)

            probabilities_without_attack.extend(probs_without_attack.numpy())

            probabilities_with_attack.extend(probs_with_attack.numpy())

    return predictions_without_attack, probabilities_without_attack,predictions_with_attack, probabilities_with_attack

# Evaluate model
predictions_without_attack, probabilities_without_attack,predictions_with_attack, probabilities_with_attack= evaluate_model(model_classifier, dataloader_attack)

KeyboardInterrupt: 

In [ ]:
print(len(probabilities_without_attack))

50


In [ ]:
#This will give the results for mean differences for top 5, top 10 , top 7 and so on change the values of k
top_k = 15
differences = []
for note_index in range(0,len(probabilities_with_attack)):
   sorted_indices_without_attack = sorted(range(len(probabilities_without_attack[note_index])), key=lambda i: probabilities_without_attack[note_index][i])
   sorted_indices_without_attack.reverse()
   sorted_indices_with_attack = sorted(range(len(probabilities_with_attack[note_index])), key=lambda i: probabilities_with_attack[note_index][i])
   sorted_indices_with_attack.reverse()
   A = set(sorted_indices_without_attack[0:top_k])
   B = set(sorted_indices_with_attack[0:top_k])
   num_matches = len(A.difference(B))
   differences.append(num_matches)

print(np.mean(differences))

1.88


960
960
1.0


False

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0.],